In [1]:
cfg_id_column = 0
cfg_title_column = 2
cfg_description_column = 13
cfg_depend_column = 4
cfg_first_valid_row = 1
cfg_project_prefix_cfid = 21
cfg_rmtracker_id = 3
cfg_rmtitle_cfid = 16
cfg_enable_duration = True
cfg_enable_assignee = True
cfg_duration_column = 9
cfg_assignee_column = 11
cfg_enable_collaborators = True
cfg_collaborators_column = 12
cfg_enable_workload = True
cfg_workload_column = 10
cfg_enable_otherdep = True
cfg_otherdep_column = 5
cfg_enable_startdate = True
cfg_startdate_column = 14

In [2]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

In [4]:
if ignore_cert:
    redmine = Redmine(rm_server_url,key=rm_key_txt, requests={'verify': False})
else:
    redmine = Redmine(rm_server_url,key=rm_key_txt)

projects = redmine.project.all()

In [5]:
print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

my_project = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

Proyectos:
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
	 est  	|  EST
	 gris-gregor  	|  GRIS@GREGOR
	 datalog  	|  Datalog
	 datalogger-update  	|  Actualización datalogger GRIS 
	 licit-20-006  	|  LICIT 20-006
	 mical  	|  MICAL
	 mcao-bench  	|  MCAO-Bench
	 mcao-aiv  	|  MCAO-AIV
	 mical-gris  	|  MICAL-GRIS
	 solarnet-ifu  	|  Solarnet IFU
	 sw-tools  	|  SW Too

Ahora procedemos a cargar el fichero

In [6]:
file_path = "input_dsm.ods"
data = get_data(file_path)['_RqDep']
tasks = {}
print("file loaded")

file loaded


In [7]:
i = 0
tasks = {}
for row in data:
    # Let's skip the first row
    if (i >= cfg_first_valid_row):
        thisTsk = {}
        if (len(row) > cfg_title_column) and (len(row[cfg_title_column]) > 0):
            thisTsk['rmid'] = 0
            thisTsk['rmident'] = ''
            thisTsk['row'] = i
            thisTsk['ident'] = row[cfg_id_column].strip()
            thisTsk['title'] = row[cfg_title_column].strip()
            if (len(row) > cfg_description_column):
                thisTsk['description'] = row[cfg_description_column].strip()
            else:
                thisTsk['description'] = None
            thisTsk['dependences'] = []
            if (len(row) > cfg_depend_column):
                tmp = row[cfg_depend_column].strip()
                if (tmp != ''):
                    thisTsk['dependences'] = tmp.split(' ')
                                
            if (cfg_enable_otherdep):
                thisTsk['otherdep'] = []
                if (len(row) > cfg_otherdep_column):
                    tmp = row[cfg_otherdep_column].strip()
                    if (tmp != ''):
                        thisTsk['otherdep'] = tmp.split(' ')
            
            #print(thisTsk)
            tasks[thisTsk['ident']] = thisTsk
            print(thisTsk['ident'])
            
            if (cfg_enable_duration):
                if (len(row) > cfg_duration_column):
                    tmp = row[cfg_duration_column]
                    if (tmp is not None) and (tmp != ''):
                        thisTsk['duration'] = tmp
                    
            if (cfg_enable_assignee):
                if (len(row) > cfg_assignee_column):
                    tmp = row[cfg_assignee_column].strip()
                    if (tmp is not None) and (tmp != ''):
                        print("assignee: |"+tmp+"|")
                        thisTsk['assignee'] = tmp

            if (cfg_enable_startdate):
                if (len(row) > cfg_startdate_column):
                    tmp = row[cfg_startdate_column]
                    if (tmp is not None) and (tmp != ''):
                        print("start date: |"+str(tmp)+"|")
                        thisTsk['startdate'] = tmp

            if (cfg_enable_workload):
                if (len(row) > cfg_workload_column):
                    tmp = row[cfg_workload_column]
                    if (tmp is not None) and (tmp != ''):
                        thisTsk['workload'] = tmp                        
                    
                        
            if (cfg_enable_collaborators):
                thisTsk['collaborators'] = []
                if (len(row) > cfg_collaborators_column):
                    tmp = row[cfg_collaborators_column].lower().strip()
                    if (tmp is not None) and (tmp != ''):
                        print("collaborators: |"+tmp+"|")
                        thisTsk['collaborators'] = tmp.split(' ')
                        
                print(thisTsk['collaborators'])


    
    i += 1
    


A20
assignee: |fgperez|
collaborators: |horacio[50] sregalado[50]|
['horacio[50]', 'sregalado[50]']
A22
assignee: |fgperez|
collaborators: |carlosd[90] sregalado[90]|
['carlosd[90]', 'sregalado[90]']
A61
assignee: |sregalado|
collaborators: |horacio[25] jquintero[50]|
['horacio[25]', 'jquintero[50]']
A21
assignee: |sregalado|
collaborators: |horacio[25] jquintero[50]|
['horacio[25]', 'jquintero[50]']
A23
assignee: |carlosd|
collaborators: |horacio[25] sregalado[90] jquintero[50]|
['horacio[25]', 'sregalado[90]', 'jquintero[50]']


In [8]:
print(tasks.keys())

dict_keys(['A20', 'A22', 'A61', 'A21', 'A23'])


Vamos a buscar las mismas tareas al Redmine, si es posible encontrarlas

In [9]:
print(tasks)

{'A20': {'rmid': 0, 'rmident': '', 'row': 1, 'ident': 'A20', 'title': 'Montaje Cámara 770 en AIV', 'description': 'Montaje de la cámara de 770nm en AIV para poder validarla, incluyendo otros elementos del banco de pruebas.', 'dependences': [], 'otherdep': [], 'duration': 24, 'assignee': 'fgperez', 'workload': 90, 'collaborators': ['horacio[50]', 'sregalado[50]']}, 'A22': {'rmid': 0, 'rmident': '', 'row': 2, 'ident': 'A22', 'title': 'Montaje de Cámara 770 + montura en GREGOR', 'description': 'Montaje de la cámara de 770nm en GREGOR, incluyendo montura.  Alineado de la cámara incluído, y conjunto de elementos necesarios para hacer una toma de imágenes individuales en 770nm con la red de difracción nueva.', 'dependences': ['A21'], 'otherdep': [], 'duration': 8, 'assignee': 'fgperez', 'workload': 90, 'collaborators': ['carlosd[90]', 'sregalado[90]']}, 'A61': {'rmid': 0, 'rmident': '', 'row': 3, 'ident': 'A61', 'title': 'Prueba de Cámara 770 + Polarímetro 650-1032 en AIV', 'description': 'E

In [10]:
# Buscamos el prefijo del proyecto
prjPrefix = my_project.identifier
lenPrjPrefix = len(prjPrefix)
# Recorreremos los identificadores de las tareas cargadas del ODS para ver si alguno se corresponde con un identificador válido de proyecto
for tskey in tasks.keys():
    thisRmTsk = None
    print("Tarea nueva",tskey)
    thisRmTsk = redmine.issue.create(project_id = my_project.id,
        tracker_id = cfg_rmtracker_id,
        description = tasks[tskey]['description'],
        subject = tasks[tskey]['title']
    )
    tasks[tskey]['rmid'] = thisRmTsk.id
    tasks[tskey]['rmtsk'] = thisRmTsk


Tarea nueva A20
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
Tarea nueva A22
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advance

In [11]:
print(dir(my_project))

['created_on', 'description', 'enabled_modules', 'files', 'homepage', 'id', 'identifier', 'inherit_members', 'is_public', 'issue_categories', 'issues', 'memberships', 'name', 'news', 'parent', 'status', 'time_entries', 'time_entry_activities', 'trackers', 'updated_on', 'versions', 'wiki_pages']


In [12]:
for tskey in tasks.keys():
    print("id",tskey,"rmident",tasks[tskey]['rmid'])

id A20 rmident 1290
id A22 rmident 1291
id A61 rmident 1292
id A21 rmident 1293
id A23 rmident 1294


In [13]:
# Recorreremos de nuevo los identificadores de las tareas cargadas del ODS para crear/modificar sus relaciones
for tskey in tasks.keys():
    odsDependences = tasks[tskey]['dependences']
    print("id",tskey,odsDependences)
    rmDependences = []
    for dpd in odsDependences:
        if (dpd != ''):
            thisRmId = tasks[dpd]['rmid']
            print(thisRmId)
            rmDependences.append(thisRmId)
        
    tasks[tskey]['rmdep'] = rmDependences
    

id A20 []
id A22 ['A21']
1293
id A61 ['A21']
1293
id A21 ['A20']
1290
id A23 ['A22']
1291


In [14]:
if (cfg_enable_otherdep):
    for tskey in tasks.keys():
        odsOtherDep = tasks[tskey]['otherdep']
        print("id",tskey,odsOtherDep)
        rmOtherDep = []
        for dpd in odsOtherDep:
            if (dpd != ''):
                thisRmId = tasks[dpd]['rmid']
                print(thisRmId)
                rmOtherDep.append(thisRmId)

        tasks[tskey]['rmotherdep'] = rmOtherDep    

id A20 []
id A22 []
id A61 []
id A21 []
id A23 []


In [15]:
# Ahora buscamos las dependencias ya existentes en el Redmine y las restamos de las relaciones a añadir
for tskey in tasks.keys():
    print("id",tskey,tasks[tskey]['dependences'],tasks[tskey]['rmdep'])
    thisTskRmId = tasks[tskey]['rmid']
    thisTskRelations = tasks[tskey]['rmtsk'].relations
    thisTskInputRelations = list(filter(lambda x: x.issue_to_id == thisTskRmId and x.relation_type=='precedes', thisTskRelations))
    print("Preexistent:", thisTskInputRelations)
    for rel in thisTskInputRelations:
        # La relación está a punto de ser añadida, cancelamos su creación
        if rel.issue_to_id in tasks[tskey]['rmdep']:
            tasks[tskey]['rmdep'].remove(rel.issue_to_id)
            print("En",tskey,"Borro",rel.issue_to_id,"de",tasks[tskey]['rmdep'])

        if rel.issue_to_id in tasks[tskey]['rmotherdep']:
            tasks[tskey]['rmotherdep'].remove(rel.issue_to_id)
            print("En",tskey,"Borro",rel.issue_to_id,"de",tasks[tskey]['rmotherdep'])               
        
        else:
            # La relación ya no existe, hay que borrarla
            redmine.issue_relation.delete(rel.id)
            
    # Ahora añadimos las relaciones restantes
    for relId in tasks[tskey]['rmdep']:
        rel = redmine.issue_relation.create(
            issue_id=relId,
            issue_to_id=thisTskRmId,
            relation_type='precedes',
            delay=0
        )

            


id A20 [] []
Preexistent: []
id A22 ['A21'] [1293]
Preexistent: []
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
id A61 ['A21'] [1293]
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advis

In [16]:
# Ahora buscamos las dependencias ya existentes en el Redmine y las restamos de las relaciones a añadir
for tskey in tasks.keys():
    print("id",tskey,tasks[tskey]['otherdep'],tasks[tskey]['rmotherdep'])
    thisTskRmId = tasks[tskey]['rmid']            
    # Ahora añadimos las relaciones restantes
    for relId in tasks[tskey]['rmotherdep']:
        rel = redmine.issue_relation.create(
            issue_id=relId,
            issue_to_id=thisTskRmId,
            relation_type='precedes',
            delay=0
        )


id A20 [] []
id A22 [] []
id A61 [] []
id A21 [] []
id A23 [] []


In [17]:
members = my_project.memberships
team = {}
for m in members:
    thisUser = redmine.user.get(m.user.id)
    team[thisUser.login] = thisUser
    
print(team.keys())


C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warn

In [18]:
if (cfg_enable_duration):    
    for tskey in tasks.keys():
        if 'duration' in tasks[tskey].keys():
            #print("------------>",tasks[tskey]['duration'])
            tasks[tskey]['rmtsk'].estimated_hours = tasks[tskey]['duration']
            tasks[tskey]['rmtsk'].save()


C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warn

In [19]:
if (cfg_enable_assignee):
    for tskey in tasks.keys():
        print(tasks[tskey]['ident'])
        if ('assignee' in tasks[tskey].keys()):
            print(tasks[tskey]['assignee'])
            tasks[tskey]['rmtsk'].assigned_to_id = team[tasks[tskey]['assignee']].id
            print(tasks[tskey]['rmtsk'])
            tasks[tskey]['rmtsk'].save()

A20
fgperez
Montaje Cámara 770 en AIV
A22
fgperez
Montaje de Cámara 770 + montura en GREGOR
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
A61
sregalado
Prueba de Cámara 770 + Polarímetro 650-1032 en AIV
A21
sregalado
Pruebas Cámara 770 en AIV
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

In [20]:
import datetime

if (cfg_enable_startdate):
    for tskey in tasks.keys():
        print(tasks[tskey]['ident'])
        if ('startdate' in tasks[tskey].keys()):
            print(tasks[tskey]['startdate'])
            tasks[tskey]['rmtsk'].start_date = tasks[tskey]['startdate']
            if (hasattr(tasks[tskey]['rmtsk'], 'due_date')):
                if (tasks[tskey]['rmtsk'].due_date < tasks[tskey]['startdate']):
                    tasks[tskey]['rmtsk'].due_date = tasks[tskey]['startdate']
            else:
                tasks[tskey]['rmtsk'].due_date = tasks[tskey]['startdate']

            print(tasks[tskey]['rmtsk'],tasks[tskey]['rmtsk'].start_date,tasks[tskey]['rmtsk'].due_date)
                
            tasks[tskey]['rmtsk'].save()

A20
A22
A61
A21
A23


In [21]:
for tskey in tasks.keys():
    depstr = ','.join(map(str,tasks[tskey]['rmdep']))
    print(depstr)
    depCf = tasks[tskey]['rmtsk'].custom_fields.get(rm_cfield_dep).value
    print(depCf)
    if 'workload' not in tasks[tskey].keys():
        tasks[tskey]['workload'] = 90
        
    wload = tasks[tskey]['workload']        
    print(wload)
    wloadCf = tasks[tskey]['rmtsk'].custom_fields.get(rm_cfield_wload).value
    print(wloadCf)
    redmine.issue.update(resource_id=tasks[tskey]['rmtsk'].id,
                         custom_fields=[{'id': rm_cfield_dep,'value': depstr},{'id': rm_cfield_wload,'value': wload}]
                        )




90
100
1293

90
100
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
1293

90
100
1290

90
100
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/e

In [22]:
if (cfg_enable_collaborators):
    for tskey in tasks.keys():
        if ('collaborators' in tasks[tskey].keys()):
            odsCollaborators = tasks[tskey]['collaborators']
            print("collaborators",tskey,odsCollaborators)
            rmCollaborators = []
            cptask = {}
            for col in odsCollaborators:
                if (col != ''):
                    if '[' in col:
                        colabtmp = col.split('[')
                        colab = colabtmp[0]
                        workload = int(colabtmp[1].split(']')[0])
                        cptask[colab] = workload
                        print(cptask)
                    
                    else:
                        colab = col
                    
                    colRmId = team[colab].id
                    print(team[colab].login)
                    rmCollaborators.append(colRmId)

            print(rmCollaborators)
            tasks[tskey]['rmcol'] = rmCollaborators
            collabCf = tasks[tskey]['rmtsk'].custom_fields.get(rm_cfield_collaborators).value
            print(collabCf)
            redmine.issue.update(resource_id=tasks[tskey]['rmtsk'].id,
                                 custom_fields=[{'id': rm_cfield_collaborators,'value': rmCollaborators}]
                                )
            
            if len(cptask.keys())>0:
                wload = tasks[tskey]['workload']
                # First we copy the issue as the first child
                copiedTsk = redmine.issue.create(project_id = my_project.id,
                    tracker_id = cfg_rmtracker_id,
                    subject = tasks[tskey]['rmtsk'].subject,
                    description = tasks[tskey]['rmtsk'].description,
                    assigned_to_id = tasks[tskey]['rmtsk'].assigned_to_id,
                    parent_issue_id = tasks[tskey]['rmtsk'].id,
                    estimated_hours = tasks[tskey]['rmtsk'].estimated_hours,
                    done_ratio = tasks[tskey]['rmtsk'].done_ratio,                                           
                    custom_fields=[
                            {'id': cfg_rmtitle_cfid,'value': tasks[tskey]['title']},{'id': rm_cfield_wload,'value': wload}
                        ]
                    )
                # Ahora añadimos las relaciones restantes
                for relId in tasks[tskey]['rmdep']+tasks[tskey]['rmotherdep']:
                    rel = redmine.issue_relation.create(
                        issue_id=relId,
                        issue_to_id=copiedTsk.id,
                        relation_type='precedes',
                        delay=0
                    )
                
                
                # Then we proceed to do a copy for each "other assignee in the collaborators list"
                for col in cptask.keys():
                    wload = cptask[col]
                    copiedTsk = redmine.issue.create(project_id = my_project.id,
                        tracker_id = cfg_rmtracker_id,
                        subject = tasks[tskey]['rmtsk'].subject,
                        description = tasks[tskey]['rmtsk'].description,
                        assigned_to_id = team[col].id,
                        parent_issue_id = tasks[tskey]['rmtsk'].id,
                        estimated_hours = tasks[tskey]['rmtsk'].estimated_hours,
                        done_ratio = tasks[tskey]['rmtsk'].done_ratio,          
                        custom_fields=[
                                {'id': cfg_rmtitle_cfid,'value': tasks[tskey]['title']},{'id': rm_cfield_wload,'value': wload}
                            ]
                        )                

                    # Ahora añadimos las relaciones restantes
                    for relId in tasks[tskey]['rmdep']+tasks[tskey]['rmotherdep']:
                        rel = redmine.issue_relation.create(
                            issue_id=relId,
                            issue_to_id=copiedTsk.id,
                            relation_type='precedes',
                            delay=0
                        )



collaborators A20 ['horacio[50]', 'sregalado[50]']
{'horacio': 50}
horacio
{'horacio': 50, 'sregalado': 50}
sregalado
[37, 50]
[]
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Txinto Vaz\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minero.ll.iac.es'. Adding c

In [23]:
print(rm_cfield_wload)

32


In [24]:
print("done")

done
